In [1]:
import pandas as pd

In [51]:
test = pd.read_parquet('../MADOC/reddit-technology/sample_1/technology_sample_1.parquet')
test.head()

,post_id,publish_date,user_id,parent_id,parent_user_id,content,url,language,interaction_type,platform,community,strict_filter,sentiment_vader,sentiment_textblob,subjectivity_textblob,toxicity_toxigen
4543776,4061545e-d376-5e38-8c44-deff8369bd71,1467450928,baf06d56-cd06-50dc-a06d-1a068c469893,None,None,"Didn't know that, going to check the purchase ...",None,English,COMMENT,reddit,technology,None,0.4926,0.250000,0.200000,0.000640
4543777,6bfea03d-66ce-5e67-94b9-ee563afd24ee,1467450953,f71710ca-2f09-5042-991f-316321da8aec,None,None,You can get things on the bottles that only po...,None,English,COMMENT,reddit,technology,None,0.2263,0.150000,0.550000,0.001062
4543778,82ffae9d-3807-502c-bff3-66a86b57ae39,1467450975,ea593194-3782-55e4-9124-d46495433ffa,None,None,HBO is not cable. It is a channel that is broa...,None,English,COMMENT,reddit,technology,None,0.5023,0.125000,0.205128,0.000880
4543779,e4e36b17-fa67-5ff5-a2ed-b849f0422815,1467450989,48d2ad45-1028-5c0d-b596-1475ceede7bd,c6006237-9621-551b-8c2c-e9bbbd551aad,8560bee3-1b72-550a-9c0b-d47f9caf2b02,Just to be 100% clear: we are not certain he i...,None,English,COMMENT,reddit,technology,None,0.1989,-0.002381,0.351587,0.000948
4543780,f782843d-e515-521b-8875-d8d11f7c7689,1467451039,ad176433-9095-5dc8-a070-8bdc91ac4a59,None,None,Bartenders are the most sensitive people I've ...,None,English,COMMENT,reddit,technology,None,-0.9030,-0.050000,0.600000,0.005992


In [52]:
test['to_count']=1

In [53]:
test.sort_values(by='publish_date', inplace=True)

In [54]:
from datetime import datetime, timezone

def days_between(d1, d2):
    d1 = datetime.fromtimestamp(d1,tz=timezone.utc)
    d2 = datetime.fromtimestamp(d2,tz=timezone.utc)
    return abs((d2 - d1).days)

days_between(1470042699, 1467451039)

30

In [70]:
test['parent_user_id'] = test['parent_user_id'].fillna('NAN')

test_ones = test[~test.duplicated(subset=['user_id', 'parent_user_id'], keep=False)][['user_id', 'parent_user_id', 'publish_date']]
test_multiples = test[test.duplicated(subset=['user_id', 'parent_user_id'], keep=False)][['user_id', 'parent_user_id', 'publish_date']]
# Create bidirectional user pairs
test_ones['user_pair'] = test.apply(
    lambda row: tuple(sorted([row['user_id'], row['parent_user_id']])), axis=1
)

test_multiples['user_pair'] = test.apply(
    lambda row: tuple(sorted([row['user_id'], row['parent_user_id']])), axis=1
)


test_ones['days'] = 0  # Assign 0 days for unique pairs

# Sort by user pair and publish_date
test_multiples = test_multiples.sort_values(['user_pair', 'publish_date'])

# Calculate previous timestamp for each bidirectional pair
test_multiples['prev_publish_date'] = test_multiples.groupby('user_pair')['publish_date'].shift(1)

# Calculate days between successive interactions
test_multiples['days'] = test_multiples.apply(
    lambda row: days_between(row['prev_publish_date'], row['publish_date'])
    if pd.notna(row['prev_publish_date']) else 0, 
    axis=1
)

test = pd.concat([test_ones, test_multiples], ignore_index=True)
# Check the results
print("Days column statistics:")
print(test['days'].describe())
print(f"\nRows with days > 0: {(test['days'] > 0).sum()}")
print(f"Total rows: {len(test)}")


Days column statistics:
count    138190.000000
mean          0.846841
std           2.934031
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          29.000000
Name: days, dtype: float64

Rows with days > 0: 19468
Total rows: 138190


In [71]:
test.sort_values(by='days', ascending=True, inplace=True)
test

,user_id,parent_user_id,publish_date,user_pair,days,prev_publish_date
86765,75ef3ae5-344a-5312-bdec-18b7a244ac12,NAN,1468594805,"(75ef3ae5-344a-5312-bdec-18b7a244ac12, NAN)",0,1.468556e+09
86781,75fd3699-1111-5775-a68f-7a082cb45a28,NAN,1468543367,"(75fd3699-1111-5775-a68f-7a082cb45a28, NAN)",0,1.468542e+09
86780,75fd3699-1111-5775-a68f-7a082cb45a28,NAN,1468541545,"(75fd3699-1111-5775-a68f-7a082cb45a28, NAN)",0,1.468541e+09
86779,75fd3699-1111-5775-a68f-7a082cb45a28,NAN,1468541067,"(75fd3699-1111-5775-a68f-7a082cb45a28, NAN)",0,1.468540e+09
86778,75fd3699-1111-5775-a68f-7a082cb45a28,NAN,1468540460,"(75fd3699-1111-5775-a68f-7a082cb45a28, NAN)",0,1.468540e+09
...,...,...,...,...,...,...
98862,96ead6db-a9de-52f1-8041-022cf964bc74,NAN,1469887325,"(96ead6db-a9de-52f1-8041-022cf964bc74, NAN)",28,1.467463e+09
123659,dac8a55b-70d8-51bc-82cb-306ca4d7bd9e,NAN,1470018897,"(NAN, dac8a55b-70d8-51bc-82cb-306ca4d7bd9e)",28,1.467558e+09
88286,7a235354-a2ee-5f58-a07c-683e3484f395,NAN,1469984260,"(7a235354-a2ee-5f58-a07c-683e3484f395, NAN)",28,1.467550e+09
108635,b22de110-2988-5e83-9a75-4c21752944f3,NAN,1469992595,"(NAN, b22de110-2988-5e83-9a75-4c21752944f3)",29,1.467479e+09


In [58]:
test_ones = test[~test.duplicated(subset=['user_id', 'parent_user_id'], keep=False)][['user_id', 'parent_user_id', 'publish_date']]
test_ones.head()

,user_id,parent_user_id,publish_date
4543776,baf06d56-cd06-50dc-a06d-1a068c469893,None,1467450928
4543779,48d2ad45-1028-5c0d-b596-1475ceede7bd,8560bee3-1b72-550a-9c0b-d47f9caf2b02,1467450989
4543780,ad176433-9095-5dc8-a070-8bdc91ac4a59,None,1467451039
4543781,0f79750a-5353-5836-8205-b5b56da68daa,None,1467451083
4543784,b1a6b159-61a9-59d1-a906-6900bad05165,None,1467451188


In [ ]:
test_ones['days'] =

In [54]:
test.groupby(['user_id', 'parent_user_id','publish_date']).count()

post_id  \
user_id                              parent_user_id                       publish_date            
000e16a6-ce6e-5bbf-9e6a-44d50d54fb54 4e7d5708-0296-5627-a3b8-d485e76e5524 1468366684          1   
001641b7-ea69-5eab-98a3-cbc5d76ce58e 22284564-b05a-5526-8555-d15911ec68d9 1469049758          1   
001b6382-7d2b-54b5-912b-cc2711416685 43060541-1bac-57d3-abd0-44a70d74ad7c 1467494206          1   
                                     e8a0389a-9bb2-50fd-b4ca-4078b99c1c09 1468075590          1   
001de577-6493-5e59-86d4-63379f9738a6 ab2dc0cf-ea87-5bc8-ad4b-357fd5779bbe 1468761217          1   
...                                                                                         ...   
ffe20208-ddf3-58da-87e1-a9e4b1d55e8a be37aa86-5c27-562f-ac96-5e20bab2eb71 1467998348          1   
ffe53449-f94d-5684-afc5-99515503051d ccb3ca39-c842-5348-91fb-cfb7cb99efb1 1468096613          1   
fffce685-1077-5a5a-8638-e93461e30552 165f5f1c-0408-5e99-9331-47ddb442888a 1469463188          1   
fffee915-0727-5b37-9212-c9668baa72a4 9c1c408c-13c5-5f84-b3b6-38f2e42743a5 1469805656          1   
fffeff9f-b2eb-5fe4-b8a0-421dc1ff1a26 c4bd4801-574f-5e08-9200-6b0a743011e8 1468587388          1   

                                                                                        parent_id  \
user_id                              parent_user_id                       publish_date              
000e16a6-ce6e-5bbf-9e6a-44d50d54fb54 4e7d5708-0296-5627-a3b8-d485e76e5524 1468366684            1   
001641b7-ea69-5eab-98a3-cbc5d76ce58e 22284564-b05a-5526-8555-d15911ec68d9 1469049758            1   
001b6382-7d2b-54b5-912b-cc2711416685 43060541-1bac-57d3-abd0-44a70d74ad7c 1467494206            1   
                                     e8a0389a-9bb2-50fd-b4ca-4078b99c1c09 1468075590            1   
001de577-6493-5e59-86d4-63379f9738a6 ab2dc0cf-ea87-5bc8-ad4b-357fd5779bbe 1468761217            1   
...                                                                                           ...   
ffe20208-ddf3-58da-87e1-a9e4b1d55e8a be37aa86-5c27-562f-ac96-5e20bab2eb71 1467998348            1   
ffe53449-f94d-5684-afc5-99515503051d ccb3ca39-c842-5348-91fb-cfb7cb99efb1 1468096613            1   
fffce685-1077-5a5a-8638-e93461e30552 165f5f1c-0408-5e99-9331-47ddb442888a 1469463188            1   
fffee915-0727-5b37-9212-c9668baa72a4 9c1c408c-13c5-5f84-b3b6-38f2e42743a5 1469805656            1   
fffeff9f-b2eb-5fe4-b8a0-421dc1ff1a26 c4bd4801-574f-5e08-9200-6b0a743011e8 1468587388            1   

                                                                                        content  \
user_id                              parent_user_id                       publish_date            
000e16a6-ce6e-5bbf-9e6a-44d50d54fb54 4e7d5708-0296-5627-a3b8-d485e76e5524 1468366684          1   
001641b7-ea69-5eab-98a3-cbc5d76ce58e 22284564-b05a-5526-8555-d15911ec68d9 1469049758          1   
001b6382-7d2b-54b5-912b-cc2711416685 43060541-1bac-57d3-abd0-44a70d74ad7c 1467494206          1   
                                     e8a0389a-9bb2-50fd-b4ca-4078b99c1c09 1468075590          1   
001de577-6493-5e59-86d4-63379f9738a6 ab2dc0cf-ea87-5bc8-ad4b-357fd5779bbe 1468761217          1   
...                                                                                         ...   
ffe20208-ddf3-58da-87e1-a9e4b1d55e8a be37aa86-5c27-562f-ac96-5e20bab2eb71 1467998348          1   
ffe53449-f94d-5684-afc5-99515503051d ccb3ca39-c842-5348-91fb-cfb7cb99efb1 1468096613          1   
fffce685-1077-5a5a-8638-e93461e30552 165f5f1c-0408-5e99-9331-47ddb442888a 1469463188          1   
fffee915-0727-5b37-9212-c9668baa72a4 9c1c408c-13c5-5f84-b3b6-38f2e42743a5 1469805656          1   
fffeff9f-b2eb-5fe4-b8a0-421dc1ff1a26 c4bd4801-574f-5e08-9200-6b0a743011e8 1468587388          1   

                                                                                        url  \
user_id                              parent_user_id                       publish_date        
000e16a6

In [39]:
datetime.fromtimestamp(test['publish_date'].min(), tz=timezone.utc), datetime.fromtimestamp(test['publish_date'].max(), tz=timezone.utc)

(datetime.datetime(2016, 7, 2, 9, 15, 28, tzinfo=datetime.timezone.utc),
 datetime.datetime(2016, 8, 1, 9, 14, 53, tzinfo=datetime.timezone.utc))

In [37]:
test.groupby(['user_id', 'parent_user_id'])['publish_date'].diff().isna().sum()

137605

In [ ]:
# Sort by user pairs and publish_date
test = test.sort_values(['user_id', 'parent_user_id', 'publish_date'])

# Calculate previous timestamp for each user pair
test['prev_publish_date'] = test.groupby(['user_id', 'parent_user_id'])['publish_date'].shift(1)

# Calculate days between current and previous interaction
test['round'] = test.apply(
    lambda row: days_between(row['prev_publish_date'], row['publish_date'])
    if pd.notna(row['prev_publish_date']) else 0, 
    axis=1
)
test

,post_id,publish_date,user_id,parent_id,parent_user_id,content,url,language,interaction_type,platform,community,strict_filter,sentiment_vader,sentiment_textblob,subjectivity_textblob,toxicity_toxigen,round,round_days,prev_publish_date
4673444,76e44c17-751a-5055-9548-83156ef01ea1,1469830697,0003dc69-5da7-51af-bfd5-b810d6ab5d94,None,None,"The toppest of men work on that ""OS"" for sure ...",None,English,COMMENT,reddit,technology,None,0.7906,0.370833,0.668056,0.002769,0,0.0,NaN
4673456,74c87237-72df-525e-a140-8c9e4ec12b0c,1469831025,0003dc69-5da7-51af-bfd5-b810d6ab5d94,None,None,Doin' God's work son. Unfortunate you had to d...,None,English,COMMENT,reddit,technology,None,-0.4588,-0.500000,1.000000,0.002222,0,0.0,NaN
4673599,b3fa80fe-0ec6-5f00-b19c-13ae5e79f3be,1469833540,0003dc69-5da7-51af-bfd5-b810d6ab5d94,None,None,Must be a store by store basis on that last pa...,None,English,COMMENT,reddit,technology,None,0.0000,0.000000,0.066667,0.000675,0,0.0,NaN
4680634,1d674365-be84-5a91-a769-61dbd4cf1271,1469990812,0003f604-ca3b-571d-8c4e-078485c68c58,None,None,Certain lines of the nyc subway system have go...,None,English,COMMENT,reddit,technology,None,0.8020,0.388095,0.723810,0.000734,0,0.0,NaN
4607518,969a566e-424e-538e-be8a-733be7e4ee30,1468682526,00059f28-acc6-55aa-8d04-91c777ffa781,None,None,What probably happened is that they were playi...,None,English,COMMENT,reddit,technology,None,0.4639,-0.057500,0.432500,0.000938,0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4653782,299d90a9-3dd8-5ebb-ab03-e0ed5838eb05,1469554270,fffbd4f1-c74b-5cdf-9ac8-3def61da5c4c,None,None,"Right, but the girl named Ashnivas posts about...",None,English,COMMENT,reddit,technology,None,0.0000,0.285714,0.535714,0.000711,0,0.0,NaN
4607747,7adf9ace-cf99-5bb9-a04e-8cb70684c0ef,1468685422,fffbdbb1-026d-5c91-9ef0-d7aefed639ee,None,None,circlejerk and savage are my least favorite th...,None,English,COMMENT,reddit,technology,None,-0.6684,0.100000,0.700000,0.640610,0,0.0,NaN
4649316,3ca3de97-bc69-5899-abad-269fb7f1b25b,1469463188,fffce685-1077-5a5a-8638-e93461e30552,4f6f86ca-f751-5fc8-8310-1d99d2eb0266,165f5f1c-0408-5e99-9331-47ddb442888a,How much is it?,None,English,COMMENT,reddit,technology,None,0.0000,0.200000,0.200000,0.002417,0,0.0,NaN
4670072,9cc7e7d1-1566-5592-809f-030ee19a7d28,1469805656,fffee915-0727-5b37-9212-c9668baa72a4,58090060-6113-5130-bb0e-36830f326879,9c1c408c-13c5-5f84-b3b6-38f2e42743a5,"Now that the chargers have been standardised, ...",None,English,COMMENT,reddit,technology,None,-0.7650,-0.187879,0.384848,0.885388,0,0.0,NaN


In [ ]:
test[test['user_id'] == '0003dc69-5da7-51af-bfd5-b810d6ab5d94' & test['parent_user_id'] == 't2_1c4k4']  # Example to check interactions for a specific user

In [ ]:
test['round'] = test['publish_date'].apply(lambda x: days_between(x, x))

'2016-08-01 09:11:39'

In [62]:
test[test['parent_id'].isin(test['post_id'])]

,post_id,publish_date,user_id,parent_id,parent_user_id,content,url,language,interaction_type,platform,community,strict_filter,sentiment_vader,sentiment_textblob,subjectivity_textblob,toxicity_toxigen
20,79411b84-44a0-578b-9172-3cf761dd9dad,1388534840,c225ab5f-72f4-5950-908f-8a6d05a79aa0,a41e62e1-96a4-543e-b654-f367553f8703,c225ab5f-72f4-5950-908f-8a6d05a79aa0,Good!thx,None,English,COMMENT,reddit,technology,None,0.0000,0.000000e+00,0.000000,0.001617
172,be384cbf-c582-579a-a570-39e0dc4f8e47,1388536736,583b3ec8-cf80-5b77-9b03-a6d9a16c990e,8a569e50-d76a-5c69-8bdf-c70cff58aec2,583b3ec8-cf80-5b77-9b03-a6d9a16c990e,I want this car really bad...,None,English,COMMENT,reddit,technology,None,0.0772,-7.000000e-01,0.666667,0.001854
341,2a5c33d4-651a-5cfa-9d75-3d8e8fd3dc71,1388539142,e8ee99d6-1d30-55e4-ba4e-8759f7171ac3,6d174c49-bfc4-5809-a4ce-378dd5e60676,487080ea-13c2-5644-85ef-c92a90bcc735,I would like to put this in perspective... An...,None,English,COMMENT,reddit,technology,None,0.1901,0.000000e+00,0.000000,0.000612
514,3a8f4176-a396-5d68-bf96-18473881a7d5,1388540911,d5d2fd4d-5d2d-55b7-884f-853b61b04e06,319956b7-65d9-5944-91e0-b092ad9e8c71,d5d2fd4d-5d2d-55b7-884f-853b61b04e06,"Google Glass - Good, Bad, Subliminal control d...",None,English,COMMENT,reddit,technology,None,-0.1680,5.551115e-17,0.633333,0.007739
536,853f3063-d6a6-520d-89c9-058b0f84a372,1388541144,78772eb5-8b03-5075-a000-23f7f29508df,06cd3925-1798-565e-9083-ce2a7ae5c4fa,17c71d5b-ecfa-5a68-96b3-fa4f1ff78d3a,aaaaaannnd this is why fat cats in suits shoul...,None,English,COMMENT,reddit,technology,None,0.0000,0.000000e+00,0.000000,0.928943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11766112,420c0bc9-453d-518c-992d-1fd89843434a,1609458873,f5fa3900-87ed-5b9e-8825-2b63620f9ba3,80afb8d3-ee3c-595b-8f3f-ad7d39b3bede,47417c4c-a8c9-56b6-aeeb-027c58a663b2,LONG LIVE FUTURESPLASH :P,None,English,COMMENT,reddit,technology,None,0.3400,2.787879e-01,0.633333,0.000725
11766114,e2722514-5163-5bcf-990b-69da0cb74dec,1609458917,8f853502-fb08-55f5-b73b-9d6c476eb0e3,80afb8d3-ee3c-595b-8f3f-ad7d39b3bede,47417c4c-a8c9-56b6-aeeb-027c58a663b2,RIP old stick games I used to love,None,English,COMMENT,reddit,technology,None,0.6369,3.000000e-01,0.400000,0.000616
11766122,a72faac5-5b15-5aaa-92ce-84ad66bb4b3b,1609459056,88786362-ae41-5aea-952c-90714d33d0db,db79da41-ec1f-5e7e-91a9-dee7702a27f2,6d2c122c-75a0-510b-be2e-f42055051983,After the election of course,None,English,COMMENT,reddit,technology,None,0.0000,0.000000e+00,0.000000,0.000690
11766127,b2994011-896a-5d6b-9e24-581fe5dd33d4,1609459109,fbdb0692-06a1-5a5f-87d6-423c8e6ea398,b17d599f-76ac-5262-ad3f-fa9891161f7f,30741215-acbf-5a73-9720-9987102fa586,Imagine if NYC took this opportunity to set up...,None,English,COMMENT,reddit,technology,None,0.7227,2.285714e-01,0.580952,0.000650


In [29]:
for i in range(1,10):
    df = pd.read_parquet(f'/home/socio/ysocial-simulations/MADOC/reddit-technology/sample_{i}/technology_sample_{i}.parquet')
    cm=df[df['interaction_type']=='COMMENT']
    cm_e=cm[~cm['parent_user_id'].isna()].shape[0]
    perc = cm_e/cm.shape[0]*100
    print(f'sample {i} leaves {perc} % comments')
    

sample 1 leaves 17.010514173479965 % comments
sample 2 leaves 16.567198966006767 % comments
sample 3 leaves 15.774382834958361 % comments
sample 4 leaves 16.086995783580274 % comments
sample 5 leaves 17.942684403527906 % comments
sample 6 leaves 19.451457312101535 % comments
sample 7 leaves 17.456793659887328 % comments
sample 8 leaves 18.15885335627597 % comments
sample 9 leaves 17.82616124707552 % comments


In [39]:
df = pd.read_parquet('/home/socio/ysocial-simulations/MADOC/reddit-technology/sample_1/technology_sample_1.parquet')
df.head()

,post_id,publish_date,user_id,parent_id,parent_user_id,content,url,language,interaction_type,platform,community,strict_filter,sentiment_vader,sentiment_textblob,subjectivity_textblob,toxicity_toxigen
4543776,4061545e-d376-5e38-8c44-deff8369bd71,1467450928,baf06d56-cd06-50dc-a06d-1a068c469893,None,None,"Didn't know that, going to check the purchase ...",None,English,COMMENT,reddit,technology,None,0.4926,0.250000,0.200000,0.000640
4543777,6bfea03d-66ce-5e67-94b9-ee563afd24ee,1467450953,f71710ca-2f09-5042-991f-316321da8aec,None,None,You can get things on the bottles that only po...,None,English,COMMENT,reddit,technology,None,0.2263,0.150000,0.550000,0.001062
4543778,82ffae9d-3807-502c-bff3-66a86b57ae39,1467450975,ea593194-3782-55e4-9124-d46495433ffa,None,None,HBO is not cable. It is a channel that is broa...,None,English,COMMENT,reddit,technology,None,0.5023,0.125000,0.205128,0.000880
4543779,e4e36b17-fa67-5ff5-a2ed-b849f0422815,1467450989,48d2ad45-1028-5c0d-b596-1475ceede7bd,c6006237-9621-551b-8c2c-e9bbbd551aad,8560bee3-1b72-550a-9c0b-d47f9caf2b02,Just to be 100% clear: we are not certain he i...,None,English,COMMENT,reddit,technology,None,0.1989,-0.002381,0.351587,0.000948
4543780,f782843d-e515-521b-8875-d8d11f7c7689,1467451039,ad176433-9095-5dc8-a070-8bdc91ac4a59,None,None,Bartenders are the most sensitive people I've ...,None,English,COMMENT,reddit,technology,None,-0.9030,-0.050000,0.600000,0.005992


In [40]:
df[df['parent_id']==df['post_id']]

,post_id,publish_date,user_id,parent_id,parent_user_id,content,url,language,interaction_type,platform,community,strict_filter,sentiment_vader,sentiment_textblob,subjectivity_textblob,toxicity_toxigen


In [64]:
df = df.groupby(['user_id','parent_user_id']).size().reset_index(name='weight').sort_values(by='weight',ascending = False)

In [66]:
import networkx as nx

In [67]:
G = nx.from_pandas_edgelist(
df,
source='user_id',
target='parent_user_id',
edge_attr='weight'  # Set Edge Attribute to Weight Column
)

edge_count = G.number_of_edges()

print(f"  - Created {edge_count} edges (comments)")

# Normalize edge weights to [0, 1]
print("  - Normalizing edge weights...")
max_weight = max([G[u][v]['weight'] for u, v in G.edges()]) if G.edges() else 1
for u, v in G.edges():
    G[u][v]['weight'] = G[u][v]['weight'] / max_weight
    G[u][v]['raw_weight'] = G[u][v]['weight'] * max_weight  # Keep raw weight for reference

  - Created 21240 edges (comments)
  - Normalizing edge weights...


In [68]:
G['48d2ad45-1028-5c0d-b596-1475ceede7bd']['8560bee3-1b72-550a-9c0b-d47f9caf2b02']

{'weight': 0.027777777777777776, 'raw_weight': 1.0}

In [16]:
df[df['interaction_type']=='COMMENT'].shape

(128303, 16)

In [23]:
cm=df[df['interaction_type']=='COMMENT']
cm_e=cm[~cm['parent_user_id'].isna()].shape[0]

In [26]:
len(df)-len(cm)

9887

In [25]:
cm_e/cm.shape[0]*100

17.010514173479965

In [8]:
df.dropna(subset = ['parent_user_id'])

,post_id,publish_date,user_id,parent_id,parent_user_id,content,url,language,interaction_type,platform,community,strict_filter,sentiment_vader,sentiment_textblob,subjectivity_textblob,toxicity_toxigen
4543779,e4e36b17-fa67-5ff5-a2ed-b849f0422815,1467450989,48d2ad45-1028-5c0d-b596-1475ceede7bd,c6006237-9621-551b-8c2c-e9bbbd551aad,8560bee3-1b72-550a-9c0b-d47f9caf2b02,Just to be 100% clear: we are not certain he i...,None,English,COMMENT,reddit,technology,None,0.1989,-0.002381,0.351587,0.000948
4543789,ca2db6e6-faf6-538a-8f03-f8a1e6d0752e,1467451284,f0fdb913-f98e-5dee-b566-837b2c398b70,c6006237-9621-551b-8c2c-e9bbbd551aad,8560bee3-1b72-550a-9c0b-d47f9caf2b02,Fuck apple they are useless.,None,English,COMMENT,reddit,technology,None,-0.7430,-0.450000,0.400000,0.978420
4543791,55c1f516-34b5-5eed-97ee-2eee4ec7d6ff,1467451471,cb5438f3-9c98-5a50-aa3c-6608f60c94a9,5a42bab3-e5c7-57be-997b-e0ae12ec9a66,78196d0c-6ccb-5fa1-8309-197d1d24dce1,A home that is connected to a road isn't priva...,None,English,COMMENT,reddit,technology,None,0.0000,0.000000,0.375000,0.050084
4543797,7304163a-69ba-5cc7-b8f0-45b04e5701ac,1467452037,1deb9354-cbe0-5889-b314-f4125d716e8e,c6006237-9621-551b-8c2c-e9bbbd551aad,8560bee3-1b72-550a-9c0b-d47f9caf2b02,this is a horrible article. it takes speculati...,None,English,COMMENT,reddit,technology,None,-0.5423,-1.000000,1.000000,0.089578
4543806,66275e8d-272a-5724-ab38-33382485d815,1467452500,5005c8a7-313f-55f4-b519-0630f2aa2bd9,aaa0d052-0017-5a05-b4dd-4e8a974216f7,330b7155-4d97-55ff-9811-cbd54abaa3fd,??? Why the downvotes?,None,English,COMMENT,reddit,technology,None,0.0000,0.000000,0.000000,0.002077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4681945,a259a361-7cd4-5f22-b03e-0f9d3f22e604,1470041619,ef49e99b-8214-5113-9658-f0ffc015f634,41d1c2cf-423a-5eb3-bb64-15fbca9c521d,ef49e99b-8214-5113-9658-f0ffc015f634,The Epson printer support UK team professional...,None,English,COMMENT,reddit,technology,None,0.9020,0.029167,0.387500,0.000596
4681946,160f9d4b-1674-5a4f-b3ab-def9ac8c2020,1470041654,57c9c146-7312-5e93-8dfb-3a3676878d28,29c6706e-108f-575d-9453-88aa1e84f452,5eb18aaa-9d5f-5869-b265-547a1cf9bdbd,"Authentication, Authorization, and Encryption....",None,English,COMMENT,reddit,technology,None,0.1406,-0.291111,0.502540,0.849640
4681950,65878753-35c1-59a5-b4ab-3523b72afb54,1470042067,9b0835b3-577d-5c20-a252-c84fcb937b1f,bf395d73-e650-58f8-bd92-64a1c5f72352,06a8c59c-88a8-5975-82c9-53449ecbdb60,"If they actually did that from scratch, then y...",None,English,COMMENT,reddit,technology,None,0.0000,0.178571,0.317857,0.005807
4681954,e0d794a3-357d-54eb-ade6-675c9439290f,1470042328,e0db9f1e-e152-556b-b17e-2f08f2aca805,29a98187-eb8b-5495-802e-c27e7f9933fe,3a3f9c15-95a6-5e6c-a06c-8c9a4622d04e,This means they're recording all of the audio ...,None,English,COMMENT,reddit,technology,None,-0.5423,-0.350000,0.583333,0.030689
